In [ ]:
# Campo minado

import random
import re

# Criar um objeto de tabuleiro para representar o tabuleiro do campo minado
# Fazemos isso para que quando codificarmos o jogo, possamos apenas dizer "crie um novo objeto de tabuleiro"
# e cave naquele quadro, etc.
class Board:
#1
    def __init__(self, dim_size, num_bombs):
        # usaremos esses parâmetros mais tarde, tamanho do tabuleiro e número de bombas
        self.dim_size = dim_size
        self.num_bombs = num_bombs

        # pegue o tabuleiro
        self.board = self.make_new_board()
        self.assign_values_to_board()

        # inicializamos um conjunto para acompanhar quais locais descobrimos
        # colocaremos tuplas (linha,coluna) nesses conjuntos
        self.dug = set()

#2
    def make_new_board(self):
        #construa um novo tabuleiro com base no tamanho dim e nas bombas numéricas
        # devemos construir a lista de listas aqui (ou qualquer representação que você preferir,
        # mas como temos um quadro 2-D, a lista de listas é mais natural)

        # geramos o novo tabuleiro
        board = [[None for _ in range(self.dim_size)] for _ in range(self.dim_size)]
        # quando ele for criado, será um array desse jeito, os None dizem que os espaços estão sem valor
        # [[None, None, ..., None],
        #  [None, None, ..., None],
        #  [...                  ],
        #  [None, None, ..., None]]
        # isso representa o nosso tabuleiro!

        # colocamos as bombas
        bombs_planted = 0 #começamos dizendo que nenhuma bomba foi adicionada
        while bombs_planted < self.num_bombs:
            loc = random.randint(0, self.dim_size**2 - 1) # retornar um número inteiro aleatório N tal que a <= N <= b
            row = loc // self.dim_size  # queremos que o número de vezes que dim_size entra em loc (nome da variável acima) para nos dizer qual linha observar
            col = loc % self.dim_size  # queremos que o restante nos diga qual índice dessa linha observar

            if board[row][col] == '*':
                # isso significa que nessa linha e coluna já existe uma bomba, logo não fazemos nada
                continue

            board[row][col] = '*' # colocamos uma bomba
            bombs_planted += 1

        return board

#3
    def assign_values_to_board(self):
        # agora que plantamos as bombas, vamos atribuir um número de 0 a 8 para todos os espaços vazios, que
         # representa quantas bombas vizinhas existem. podemos pré-calculá-los e isso nos poupará algum
         # esforço para verificar o que está acontecendo no quadro mais tarde :)
        for r in range(self.dim_size):
            for c in range(self.dim_size):
                if self.board[r][c] == '*':
                    # se já é uma bomba, não temos que calcular nada
                    continue
                self.board[r][c] = self.get_num_neighboring_bombs(r, c)

#4
    def get_num_neighboring_bombs(self, row, col):
        # vamos percorrer cada uma das posições vizinhas e somar o número de bombas
        # top left: (row-1, col-1)
        # top middle: (row-1, col)
        # top right: (row-1, col+1)
        # left: (row, col-1)
        # right: (row, col+1)
        # bottom left: (row+1, col-1)
        # bottom middle: (row+1, col)
        # bottom right: (row+1, col+1)

        # ps: precisamos ter certeza de que não saímos dos limites!!
        num_neighboring_bombs = 0
        for r in range(max(0, row-1), min(self.dim_size-1, row+1)+1):
            for c in range(max(0, col-1), min(self.dim_size-1, col+1)+1):
                if r == row and c == col:
                    # ponto original, não precisa checar
                    continue
                if self.board[r][c] == '*':
                    num_neighboring_bombs += 1

        return num_neighboring_bombs

#5
    def dig(self, row, col):
        # cave naquele local!
         # retorna True se a escavação for bem-sucedida, False se a bomba for escavada

         # alguns cenários a serem considerados:
         # acerte uma bomba -> fim do jogo
         # cavar em um local com bombas vizinhas -> terminar a escavação
         # cave em um local sem bombas vizinhas -> cave vizinhos chamando novamente a função!
        self.dug.add((row, col)) # acompanhe o que cavamos aqui

        if self.board[row][col] == '*':
            return False
        elif self.board[row][col] > 0:
            return True

        # self.board[row][col] == 0
        for r in range(max(0, row-1), min(self.dim_size-1, row+1)+1):
            for c in range(max(0, col-1), min(self.dim_size-1, col+1)+1):
                if (r, c) in self.dug:
                    continue # não precisamos cavar onde já foi cavado
                self.dig(r, c)

        # se nossa escavação inicial não atingiu uma bomba, *não deveríamos* atingir uma bomba aqui
        return True

#6
    def __str__(self):
        # esta é uma função mágica onde se você chamar print neste objeto,
         # imprimirá o que esta função retorna!
         # retorna uma string que mostra o tabuleiro para o jogador

         # primeiro vamos criar um novo array que represente o que o usuário veria
        visible_board = [[None for _ in range(self.dim_size)] for _ in range(self.dim_size)]
        for row in range(self.dim_size):
            for col in range(self.dim_size):
                if (row,col) in self.dug:
                    visible_board[row][col] = str(self.board[row][col])
                else:
                    visible_board[row][col] = ' '

        # juntamos isso em uma string
        string_rep = ''
        # obter larguras máximas de coluna para impressão
        widths = []
        for idx in range(self.dim_size):
            columns = map(lambda x: x[idx], visible_board)
            widths.append(
                len(
                    max(columns, key = len)
                )
            )

        # imprimir as strings csv
        indices = [i for i in range(self.dim_size)]
        indices_row = '   '
        cells = []
        for idx, col in enumerate(indices):
            format = '%-' + str(widths[idx]) + "s"
            cells.append(format % (col))
        indices_row += '  '.join(cells)
        indices_row += '  \n'

        for i in range(len(visible_board)):
            row = visible_board[i]
            string_rep += f'{i} |'
            cells = []
            for idx, col in enumerate(row):
                format = '%-' + str(widths[idx]) + "s"
                cells.append(format % (col))
            string_rep += ' |'.join(cells)
            string_rep += ' |\n'

        str_len = int(len(string_rep) / self.dim_size)
        string_rep = indices_row + '-'*str_len + '\n' + string_rep + '-'*str_len

        return string_rep

#7 – veja que essa função não está dentro da classe Board
def play(dim_size=10, num_bombs=10):
    # Passo 1: crie o tabuleiro e plante as bombas
    board = Board(dim_size, num_bombs)

     # Etapa 2: mostre o quadro ao usuário e pergunte onde ele deseja cavar
     # Etapa 3a: se o local for uma bomba, mostre a mensagem de fim de jogo
     # Passo 3b: se o local não for uma bomba, cave recursivamente até que um dos quadrados esteja próximo a uma bomba
     # Passo 4: repita os passos 2 e 3a/b até que não haja mais lugares para cavar e então mostre a vitória
    safe = True

    while len(board.dug) < board.dim_size ** 2 - num_bombs:
        print(board)
        # 0,0 or 0, 0 or 0,    0
        user_input = re.split(',(\\s)*', input("Onde você quer cavar? coloque uma linha, coluna: "))  # '0, 3'
        row, col = int(user_input[0]), int(user_input[-1])
        if row < 0 or row >= board.dim_size or col < 0 or col >= dim_size:
            print("Localização inválida, tente novamente.")
            continue

        # se é um valor válido, cave
        safe = board.dig(row, col)
        if not safe:
            # bomba
            break # (game over)

    # 2 maneiras de encerrar o loop, vamos verificar qual delas
    if safe:
        print("PARABÉNS!!!! VOCÊ GANHOU!")
    else:
        print("GAME OVER :(")
        # let's reveal the whole board!
        board.dug = [(r,c) for r in range(board.dim_size) for c in range(board.dim_size)]
        print(board)

if __name__ == '__main__':
    #Desafio: diminuir tamanho do campo e aumentar a quantidade de bombas
    play(dim_size=8, num_bombs=15)

   0  1  2  3  4  5  6  7  
----------------------------
0 |  |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |
4 |  |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |
----------------------------
Onde você quer cavar? coloque uma linha, coluna: 44
Localização inválida, tente novamente.
   0  1  2  3  4  5  6  7  
----------------------------
0 |  |  |  |  |  |  |  |  |
1 |  |  |  |  |  |  |  |  |
2 |  |  |  |  |  |  |  |  |
3 |  |  |  |  |  |  |  |  |
4 |  |  |  |  |  |  |  |  |
5 |  |  |  |  |  |  |  |  |
6 |  |  |  |  |  |  |  |  |
7 |  |  |  |  |  |  |  |  |
----------------------------
Onde você quer cavar? coloque uma linha, coluna: 4,4
GAME OVER :(
   0  1  2  3  4  5  6  7  
----------------------------
0 |* |1 |0 |0 |0 |0 |0 |0 |
1 |2 |2 |0 |1 |1 |1 |1 |1 |
2 |* |2 |1 |2 |* |3 |2 |* |
3 |2 |* |2 |4 |* |5 |* |3 |
4 |2 |3 |* |3 |* |5 |* |3 |
5 |1 |* |2 |2 |2 |* |5 |* 

In [ ]:
# Forca

import random
import string

# Desafio: Lista com 10 palavras em português
words = ["embrulho", "mordomo", "motor", "programação", "python", "pulseira", "monte", "teclado", "telefone", "estrutura"]

# Layout da forca para diferentes estados de vida
lives_visual_dict = {
    9: "",
    8: """
    |
    |
    |
    |
    |
    """,
    7: """
    _________
    | /
    |/
    |
    |
    |
    """,
    6: """
    _________
    | /      |
    |/
    |
    |
    |
    """,
    5: """
    _________
    | /      |
    |/      ( )
    |
    |
    |
    """,
    4: """
    _________
    | /      |
    |/      ( )
    |        |
    |
    |
    """,
    3: """
    _________
    | /      |
    |/      ( )
    |        |
    |       /
    |
    """,
    2: """
    _________
    | /      |
    |/      ( )
    |        |
    |       / \\
    |
    """,
    1: """
    _________
    | /      |
    |/      ( )
    |       \\|
    |       / \\
    |
    """,
    0: """
    _________
    | /      |
    |/      ( )
    |       \\|/
    |       / \\
    |
    """
}

def get_valid_word(words):
    word = random.choice(words) #escolhe aleatoriamente uma palavr
    while '-' in word or ' ' in word:
        word = random.choice(words)
    return word.upper()


def hangman():
    word = get_valid_word(words)
    word_letters = set(word) # letras na palavra
    alphabet = set(string.ascii_uppercase)
    used_letters = set() # letra que o usuário escolhe

    # Desafio: 9 vidas ao invés de 7
    lives = 9

    # getting user input
    while len(word_letters) > 0 and lives > 0:
        # letras usadas
        # ' '.join(['a', 'b', 'cd']) --> 'a b cd'
        print('Você tem', lives, 'vidas restantes e já usou estas letras:', ' '.join(used_letters))

        # what current word is (ie W - R D)
        word_list = [letter if letter in used_letters else '-' for letter in word]
        print(lives_visual_dict[lives])
        print('Palavra atual: ', ' '.join(word_list))

        user_letter = input('Escolha uma letra: ').upper()
        if user_letter in alphabet - used_letters:
            used_letters.add(user_letter)
            if user_letter in word_letters:
                word_letters.remove(user_letter)
            else:
                lives -= 1  # tira uma vida se errado
                print('\nSua letra,', user_letter, 'não está na palavra.')
        elif user_letter in used_letters:
            print('\nVocê já escolheu essa letra. Tente outra.')
        else:
            print('\nIsso não é uma letra válida.')

    # gets here when len(word_letters) == 0 OR when lives == 0
    if lives == 0:
        print(lives_visual_dict[lives])
        print('You are dead. A palavra era', word)
    else:
        print('YAY! Você acertou a palavra', word, '!!')

if __name__ == '__main__':
    hangman()


In [ ]:
import random

def is_win(player, opponent):
    # retorne verdadeiro (true) se o jogador ganhar
    # r > s, s > p, p > r
    if (player == 'r' and opponent == 's') or (player == 's' and opponent == 'p') \
    or (player == 'p' and opponent == 'r'):
        return True

def play():
    user = input("Qual sua escolha? 'r' para pedra, 'p' para papel, 's' para tesoura:\n")
    computer = random.choice(['r', 'p', 's'])

    if user != 'r' and user != 'p' and user != 's':
        return "Opção inválida."

    if user == computer:
        return f"Empate! Ambos escolheram '{user}'."

    # r > s, s > p, p > r
    if is_win(user, computer):
        return f"Você ganhou! {user} vence {computer}."
    else:
        return f"Você perdeu! {computer} vence {user}."

print(play())